# <font size=6 color='green'><center>**IT 496 - Introduction to Data Mining**</center></font>
### **<center>Course Project - 03<br/><br/>ICC Cricket World Cup 2023 ML Challange<center>**

 &emsp; **Group Name : <font color='#0099ff'>DataForce</font>** <br/>
 &emsp; **Group ID : <font color='#0099ff'>T06</font>**

### <center>Task 1</center>

**Problems**
1. Predicting the batsman who will score most runs/ hit most sixes/ make most centuries in
the tournament.
2. Predicting the batsman who will score most runs/ hit most sixes/ make most centuries in
the tournament.

---

### <center>Task 2</center>
You are required to predict the two finalist teams in the ICC Cricket World Cup 2023 and the
players (11 players for each finalist team) who are likely to be part of these teams. This task
involves team composition prediction.

---

### <center>Task 3</center>
You should predict the winner of the ICC Cricket World Cup 2023.

---